In [66]:
!pip install lightgbm

In [67]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import lightgbm as lgb
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from lightgbm import LGBMClassifier

In [68]:
wids = pd.read_csv("training_v2.csv")
wids.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [76]:
def null_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : 'Percent of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        'Percent of Total Values', ascending=False).round(1)
        print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [77]:
null_values(wids)

Dataframe has 186 columns.
There are 175 columns that have missing values.


,Missing Values,Percent of Total Values
h1_bilirubin_max,84619,92.3
h1_bilirubin_min,84619,92.3
h1_lactate_min,84369,92.0
h1_lactate_max,84369,92.0
h1_albumin_max,83824,91.4
h1_albumin_min,83824,91.4
h1_pao2fio2ratio_min,80195,87.4
h1_pao2fio2ratio_max,80195,87.4
h1_arterial_ph_min,76424,83.3
h1_arterial_ph_max,76424,83.3


In [78]:
# Dropping columns that have over than 70% missing values. 
missing_raw = pd.DataFrame(wids.isna().sum()/wids.shape[0]).reset_index()

cols_drop = missing_raw.loc[missing_raw[0]>0.7, 'index']

wids.drop(cols_drop, axis=1, inplace=True)

In [79]:
X = wids.drop('hospital_death', axis=1)
y = wids['hospital_death']

In [80]:
X1 = pd.get_dummies(X)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2, random_state = 30)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(73370, 182) (18343, 182) (73370,) (18343,)


In [83]:
# Create lgb dataset
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# train
model_full = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=600,
                                learning_rate=0.1, n_estimators=50, reg_lambda=0.2)

model_full.fit(X_train, y_train)
y_pred = pd.Series(model_full.predict(X_test))
y_pred

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
18313    0
18314    0
18315    0
18316    0
18317    0
18318    0
18319    0
18320    0
18321    0
18322    0
18323    0
18324    0
18325    0
18326    0
18327    0
18328    0
18329    0
18330    0
18331    0
18332    0
18333    0
18334    0
18335    0
18336    1
18337    0
18338    0
18339    0
18340    0
18341    0
18342    0
Length: 18343, dtype: int64

In [96]:
scores = cross_val_score(model_full, X1, y, cv = 10, scoring = 'accuracy')
scores

array([0.92248147, 0.92749673, 0.93240297, 0.92836895, 0.92564326,
       0.925199  , 0.92607131, 0.92585323, 0.92551799, 0.92649945])

In [84]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[16611,   200],
       [ 1032,   500]], dtype=int64)

In [85]:
print("Precision Score:", precision_score(y_test, y_pred))
print("Recall Score:",recall_score(y_test, y_pred))
print("F1 Score:",f1_score(y_test, y_pred))

Precision Score: 0.7142857142857143
Recall Score: 0.3263707571801567
F1 Score: 0.44802867383512546


In [86]:
y_prob_pred = model_full.predict_proba(X_train)
y_prob_pred = y_prob_pred[:,1]
r_a_score = roc_auc_score(y_test, y_pred)
print("ROC-AUC-Score:", r_a_score)

ROC-AUC-Score: 0.6572368924797934


### Using full LightGBM model

In [88]:
# Create lgb dataset
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.1 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'mae'          # or 'mae'
params['sub_feature'] = 0.50      # feature_fraction 
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 600  # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

Starting training...
[1]	valid_0's l1: 0.152773
[2]	valid_0's l1: 0.149782
[3]	valid_0's l1: 0.146939
[4]	valid_0's l1: 0.144363
[5]	valid_0's l1: 0.142239
[6]	valid_0's l1: 0.140144
[7]	valid_0's l1: 0.138276
[8]	valid_0's l1: 0.136397
[9]	valid_0's l1: 0.134646
[10]	valid_0's l1: 0.133014
[11]	valid_0's l1: 0.131586
[12]	valid_0's l1: 0.130186
[13]	valid_0's l1: 0.128833
[14]	valid_0's l1: 0.127666
[15]	valid_0's l1: 0.126607
[16]	valid_0's l1: 0.125619
[17]	valid_0's l1: 0.124707
[18]	valid_0's l1: 0.123787
[19]	valid_0's l1: 0.12299
[20]	valid_0's l1: 0.122231
Saving model...
Starting predicting...


In [93]:
y_pred=gbm.predict(X_test)
for i in range(0,len(y_pred)):
    if y_pred[i]>=0.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0
    
y_pred = pd.Series(y_pred)

In [95]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Precision Score:", precision_score(y_test, y_pred))
print("Recall Score:",recall_score(y_test, y_pred))
print("F1 Score:",f1_score(y_test, y_pred))

y_prob_pred = model_full.predict_proba(X_train)
y_prob_pred = y_prob_pred[:,1]
r_a_score = roc_auc_score(y_test, y_pred)
print("ROC-AUC-Score:", r_a_score)

[[16704   107]
 [ 1228   304]]
Precision Score: 0.7396593673965937
Recall Score: 0.19843342036553524
F1 Score: 0.3129181677817808
ROC-AUC-Score: 0.5960342701137652
